In [1]:
# notebooks/content_based.ipynb

import pandas as pd
import numpy as np

# Load dataset
products = pd.read_csv('../data/olist_products_dataset.csv')
items = pd.read_csv('../data/olist_order_items_dataset.csv')
orders = pd.read_csv('../data/olist_orders_dataset.csv')
reviews = pd.read_csv('../data/olist_order_reviews_dataset.csv')

# Lihat 5 baris awal
products.head()

ModuleNotFoundError: No module named 'pandas'